<a href="https://colab.research.google.com/github/saira934/saira-saleem/blob/main/Copy_of_Sales_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
benroshan_ecommerce_data_path = kagglehub.dataset_download('benroshan/ecommerce-data')

print('Data source import complete.')


100%|██████████| 17.9k/17.9k [00:00<00:00, 3.30MB/s]

Extracting files...
Data source import complete.


<img style="margin: 0 auto; max-height: 300px" src="https://zipforecasting.com/assets/images/topiccluster/9-types-of-sales-analysis-techniques-for-businesses-800x400.png" width=900px>

# What is Sales Analysis

Sales analysis is mining your data to evaluate the performance of your sales team against its goals. It provides insights about the top performing and underperforming products/services, the problems in selling and market opportunities, sales forecasting, and sales activities that generate revenue.

Regular sales data analysis provides an understanding of the products that your customers are buying and helps you dissect why they are behaving in a certain way. You can also find patterns in your lead conversions and drop offs. All of these aspects enable you to optimize your sales process.

<img align="right" style="max-height:200px" src="https://www.winacc.com/wp-content/uploads/2018/09/Business-Benefits-of-the-Sales-Analysis-Report-Forming-a-Data-Backed-Marketing-Strategy-1080x600.jpg"/>

## Importance of Sales Analysis
<ul>
    <li>Make data-driven decisions instead of relying on gut instinct</li>
    <li>Find most profitable customers</li>
    <li>Get awareness on the market trends </li>
    <li>Expand your market reach</li>
</ul>

Here, I'll conduct a sales analysis on the E-Commerce dataset.

In [2]:
import numpy as np
import pandas as pd
import calendar

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [3]:
import pandas as pd

# Assuming the file is downloaded using kagglehub and stored in 'benroshan_ecommerce_data_path'
order = pd.read_csv(benroshan_ecommerce_data_path + '/List of Orders.csv')
order.head()

,Order ID,Order Date,CustomerName,State,City
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
3,B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
4,B-25605,05-04-2018,Kasheen,West Bengal,Kolkata


In [ ]:
import pandas as pd

# Assuming the file is downloaded using kagglehub and stored in 'benroshan_ecommerce_data_path'
details = pd.read_csv(benroshan_ecommerce_data_path + '/Order Details.csv')
details.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,66.0,-12.0,5,Clothing,Stole
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,168.0,-111.0,2,Electronics,Phones


In [ ]:
import pandas as pd

# Assuming the file is downloaded using kagglehub and stored in 'benroshan_ecommerce_data_path'
# The original path was '../input/ecommerce-data/Sales target.csv'
# Change it to the correct path based on your file location if it's different
target = pd.read_csv(benroshan_ecommerce_data_path + '/Sales target.csv')
target.head()

,Month of Order Date,Category,Target
0,Apr-18,Furniture,10400.0
1,May-18,Furniture,10500.0
2,Jun-18,Furniture,10600.0
3,Jul-18,Furniture,10800.0
4,Aug-18,Furniture,10900.0


# Data Pre-processing
## Data Cleaning
In this dataset, the data cleaning process will consists of:
<ol>
    <li>Changing the variables to appropriate Data types</li>
    <li>Removing Null Values</li>
</ol>

In [ ]:
# Cleaning the order dataset
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order ID      500 non-null    object
 1   Order Date    500 non-null    object
 2   CustomerName  500 non-null    object
 3   State         500 non-null    object
 4   City          500 non-null    object
dtypes: object(5)
memory usage: 22.0+ KB


In [ ]:
# Changing the Order Date variable to datetime data type
order['Order Date'] = order['Order Date'].astype('datetime64[ns]')

In [ ]:
# Checking null values
order.isnull().sum()

,0
Order ID,60
Order Date,60
CustomerName,60
State,60
City,60


In [ ]:
# Droping Null Values
order = order.dropna()
order.info()

In [ ]:
# Cleaning the detail dataset
details.info()

In [ ]:
# Null values
details.isnull().sum()

In [ ]:
# Chaning the Category and Sub-category variable to categorical data type
details['Category'] = details['Category'].astype('category')
details['Sub-Category'] = details['Sub-Category'].astype('category')
details.info()

In [ ]:
# Cleaning Target dataset
target.info()

In [ ]:
# Coverting Category variable to category data
target['Category'] = target['Category'].astype('category')
target.info()

In [ ]:
# Cleanded Details data
details.head()

In [ ]:
# Cleaned Order Data
order.head()

In [ ]:
# Cleaned Target Dataset
target.head()

Making a new dataframe containing the Amount, Profit and Quantity of the different orders. Then joining it with the Order datasets by taking Order ID as the Primary Key.

In [ ]:
profits = details.groupby('Order ID').sum().reset_index()
profits.head()

In [ ]:
df = pd.merge(order, profits)
df.head()

<hr>

# Sales Trend Analysis
Trend analysis is to find patterns in data, such as ups & downs. A “trend” is an upwards or downwards shift in a data set over time. In retail, this analysis of past trends in sales or revenue; allows to predict the future market. This analysis useful for budgeting and forecasting. Total sales of any business on a trend line may obtain some significant information.

In [ ]:
import pandas as pd

# Assuming the file is downloaded using kagglehub and stored in 'benroshan_ecommerce_data_path'
order = pd.read_csv(benroshan_ecommerce_data_path + '/List of Orders.csv')
# ... (other code to clean and prepare 'order' and 'details' dataframes)

profits = details.groupby('Order ID').sum().reset_index()
# Creating the 'df' dataframe
df = pd.merge(order, profits) # This line defines 'df'

# Now you can use 'df' in the following cells
df['Year'] = pd.DatetimeIndex(df['Order Date']).year
df['Month_Number'] = pd.DatetimeIndex(df['Order Date']).month
df['Month'] = df['Month_Number'].apply(lambda x: calendar.month_abbr[x])

year_month = df.groupby(['Year', 'Month','Month_Number']).sum().sort_values(['Year','Month_Number'])
year_month

Order ID  \
Year Month Month_Number                                                      
2018 Jan   1             B-25601B-25602B-25645B-25646B-25676B-25706B-25...   
     Feb   2             B-25677B-25738B-25769B-25770B-25771B-25772B-25...   
     Mar   3             B-25603B-25604B-25647B-25678B-25739B-25740B-25...   
     Apr   4             B-25614B-25615B-25616B-25617B-25618B-25619B-25...   
     May   5             B-25605B-25649B-25657B-25658B-25659B-25660B-25...   
     Jun   6             B-25606B-25607B-25650B-25689B-25690B-25691B-25...   
     Jul   7             B-25651B-25682B-25712B-25720B-25721B-25722B-25...   
     Aug   8             B-25608B-25652B-25653B-25683B-25713B-25744B-25...   
     Sep   9             B-25609B-25610B-25684B-25714B-25745B-25776B-25...   
     Oct   10            B-25654B-25685B-25715B-25746B-25777B-25810B-25...   
     Nov   11            B-25611B-25655B-25656B-25686B-25687B-25688B-25...   
     Dec   12            B-25612B-25613B-25717B-25718B-25719B-25748B-25...   
2019 Jan   1             B-25928B-25947B-25948B-25949B-25950B-25951B-25...   
     Feb   2             B-25929B-25990B-26012B-26013B-26014B-26015B-26...   
     Mar   3             B-25930B-25991B-25992B-25993B-25994B-26045B-26...   
     Apr   4             B-25931B-25932B-25933B-25934B-25935B-25995B-25...   
     May   5                    B-25936B-25937B-25938B-25939B-25999B-26049   
     Jun   6                                         B-25940B-26000B-26050   
     Jul   7                                         B-25941B-26001B-26051   
     Aug   8                    B-25942B-26002B-26003B-26004B-26005B-26052   
     Sep   9                           B-25943B-26006B-26007B-26008B-26053   
     Oct   10                   B-25944B-26009B-26054B-26055B-26056B-26057   
     Nov   11                                        B-25945B-26010B-26058   
     Dec   12                                        B-25946B-26011B-26059   

                                                                Order Date  \
Year Month Month_Number                                                      
2018 Jan   1             01-04-201801-04-201801-05-201801-05-201801-06-...   
     Feb   2             02-06-201802-08-201802-09-201802-09-201802-09-...   
     Mar   3             03-04-201803-04-201803-05-201803-06-201803-08-...   
     Apr   4             13-04-201815-04-201815-04-201817-04-201818-04-...   
     May   5             05-04-201805-05-201813-05-201814-05-201815-05-...   
     Jun   6             06-04-201806-04-201806-05-201814-06-201815-06-...   
     Jul   7             07-05-201807-06-201807-07-201815-07-201816-07-...   
     Aug   8             08-04-201808-05-201808-05-201808-06-201808-07-...   
     Sep   9             09-04-201809-04-201809-06-201809-07-201809-08-...   
     Oct   10            10-05-201810-06-201810-07-201810-08-201810-09-...   
     Nov   11            11-04-201811-05-201811-05-201811-06-201811-06-...   
     Dec   12            12-04-201812-04-201812-07-201812-07-201812-07-...   
2019 Jan   1             01-01-201913-01-201913-01-201913-01-201913-01-...   
     Feb   2             02-01-201902-02-201913-02-201913-02-201913-02-...   
     Mar   3             03-01-201903-02-201903-02-201903-02-201903-02-...   
     Apr   4             04-01-201904-01-201904-01-201904-01-201904-01-...   
     May   5             05-01-201905-01-201905-01-201905-01-201905-02-...   
     Jun   6                                06-01-201906-02-201906-03-2019   
     Jul   7                                07-01-201907-02-201907-03-2019   
     Aug   8             08-01-201908-02-201908-02-201908-02-201908-02-...   
     Sep   9             09-01-201909-02-201909-02-201909-02-201909-03-...   
     Oct   10            10-01-201910-02-201910-03-201910-03-201910-03-...   
     Nov   11                               11-01-201911-02-201911-03-2019   
     Dec   12                               12-01-201912-02-201912-03-2019   

        

In [ ]:
year_month = year_month.reset_index()
year_month["Color"] = np.where(year_month["Profit"]<0, 'Loss', 'Profit')
year_month_2018 = year_month[year_month['Year']==2018]
fig = px.bar(year_month_2018, x='Month_Number', y='Profit', color='Color',
             title="Monthly Profit in 2018",
             labels=dict(Month_Number="Month", Profit="Profit", Color="Results"),
             color_discrete_map={
                 'Loss': '#EC2049',
                 'Profit': '#2F9599'},
             hover_data=["Month", "Profit"],
             template='plotly_white')

fig.update_layout(yaxis_tickprefix = '₹', yaxis_tickformat = ',.2f')

fig.update_layout(
    xaxis = dict(
        tickvals = [1, 2, 3, 4, 5, 6, 7,8 ,9, 10, 11, 12],
        ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    )
)
fig.show()


In [ ]:
year_month_2019 = year_month[year_month['Year']==2019]
fig = px.bar(year_month_2019, x='Month_Number', y='Profit', color='Color',
             title="Monthly Profit in 2019",
             labels=dict(Month_Number="Month", Profit="Profit", Color="Results"),
             color_discrete_map={
                 'Loss': '#EC2049',
                 'Profit': '#2F9599'},
             hover_data=["Month", "Profit"],
             template='plotly_white')

fig.update_layout(yaxis_tickprefix = '₹', yaxis_tickformat = ',.2f')

fig.update_layout(
    xaxis = dict(
        tickvals = [1, 2, 3, 4, 5, 6, 7,8 ,9, 10, 11, 12],
        ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    )
)
fig.show()

<hr>

In [ ]:
orders_by_state = order.groupby(['State']).size().reset_index(name='Total Orders').sort_values(['Total Orders'])
orders_by_state

In [ ]:
profit_by_state = df.groupby('State').sum().reset_index().sort_values(['Profit'])
profit_by_state["Color"] = np.where(profit_by_state["Profit"]<0, 'Loss', 'Profit')


fig = px.bar(profit_by_state, x='State', y='Profit',
             color='Color', color_discrete_map={
                 'Loss': '#EC2049',
                 'Profit': '#2F9599'},
             title="Profit by State",
             labels=dict(Color="Results"),
             template='plotly_white')

# Disabling Zoom
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True

fig.update_layout(yaxis_tickprefix = '₹', yaxis_tickformat = ',.2f')

fig.update_xaxes(
        tickangle = -90,
        title_text = "States",
)


fig.show()

In [ ]:
top_customers = df.groupby('CustomerName').sum().reset_index().sort_values(['Quantity'], ascending=False).head(5)

colors = ['lightslategray',] * 5
colors[0] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=top_customers['CustomerName'],
    y=top_customers['Quantity'],
    marker_color=colors # marker color can be a single color value or an iterable,
)])
fig.update_layout(title_text='Top 5 Customers',
                  template='plotly_white')
fig.update_xaxes(title_text='Customers')
fig.update_yaxes(title_text='Total Orders')

fig.show()

In [ ]:
details_category = details.groupby('Category').sum().reset_index()
fig = px.pie(details_category, values='Quantity', names='Category', color='Category',
             color_discrete_map={'Clothing':'cyan',
                                 'Electronics':'royalblue',
                                 'Furniture':'darkblue'},
            title='Total Quantity Sold per Category')
fig.show()

In [ ]:
details_subcategory = details.groupby('Sub-Category').sum().reset_index()
fig = px.pie(details_subcategory, values='Quantity', names='Sub-Category', color='Sub-Category',
            title='Total Quantity Sold per Sub-Category')
fig.show()

In [ ]:
date_orders = order.groupby('Order Date').size().reset_index(name="Orders")
date_orders['Month'] = pd.DatetimeIndex(date_orders['Order Date']).month
date_orders['Year'] = pd.DatetimeIndex(date_orders['Order Date']).year

date_orders_2018 = date_orders[date_orders['Year']==2018]
date_orders_2019 = date_orders[date_orders['Year']==2019]

month_2018 = date_orders_2018.groupby('Month').sum().reset_index()
month_2019 = date_orders_2019.groupby('Month').sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(
    name='2018',
    x=month_2018['Month'],
    y=month_2018['Orders'],
    connectgaps=True # override default to connect the gaps
))
fig.add_trace(go.Scatter(
    name='2019',
    x=month_2019['Month'],
    y=month_2019['Orders'],
    connectgaps=True # override default to connect the gaps
))
fig.update_layout(title_text='Monthly Quantity Sold',
                 template='plotly_dark')
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Orders')
fig.update_layout(
    xaxis = dict(
        tickvals = [1, 2, 3, 4, 5, 6, 7,8 ,9, 10, 11, 12],
        ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    )
)

fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True

fig.show()

<hr>

# Customer Demographic Analysis
Customer demographics are categories of consumer populations that are relevant to a business' purposes, such as marketing and product design. The term also refers to the study of such categories in a business context.

In [ ]:
fig = px.bar(orders_by_state, y='State', x='Total Orders',
             title="Total Orders by State",
             color_discrete_sequence=["springgreen"],
             template='plotly_white')

# Disabling Zoom
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True

fig.show()

In [ ]:
#
orders_by_city = order.groupby(['City']).size().reset_index(name='Total Orders').sort_values(['Total Orders'])

fig = px.bar(orders_by_city, y='City', x='Total Orders',
             title="Total Orders by City",
             template='simple_white')

fig.layout.yaxis.tickmode='linear'
# Disabling Zoom
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True

fig.show()

The state with the highest quantity sold Madhya Pradesh, followed by Maharastra and Rajasthan. There is a biggest gap between the quantity sold in Maharastra an Rajasthan with a difference of 58 units. While in case of Cities, it is Indore and Mumbai by a very wide margin.
Chennai, Allahabad and Amritsar have the lowest quantity sold with less than 10 units sold.

<hr>

# Sales Target
A sales target is a goal set for a salesperson or sales department measured in revenue or units sold for a specific time.

In [ ]:
target_category = target.groupby('Category').max().reset_index()
details_category = details.groupby('Category').sum().reset_index()

target_category['Actual_Amount'] = details_category['Profit']

fig = go.Figure(data=[
    go.Bar(name='Target', x=target_category['Category'], y=target_category['Target'],
          marker_color='#2b2d42'),
    go.Bar(name='Actual Amount', x=target_category['Category'], y=target_category['Actual_Amount'],
          marker_color='#d90429')
])

fig.update_layout(title_text='Actual vs Target Sales',
                 template='plotly_white')

fig.update_xaxes(title_text='Categories')
fig.update_yaxes(title_text='Amount')

fig.update_layout(yaxis_tickprefix = '₹', yaxis_tickformat = ',.2f')


fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True

fig.show()

The above bar graph illustrates the Target and the Actual Amount of Profits per Quantity. None of the category surpass or even meet the target. The most disappointing Category is Furniture, which managed to have a profit of  Rs. 2298 with the target of Rs. 11.8K

<hr>

# Customer Segmentation via Cluster Analysis
Cluster analysis uses mathematical models to discover groups of similar customers based on the smallest variations among customers within each group.

### Cluster Analysis
Cluster analysis is the use of a mathematical model to discover groups of similar customers based on finding the smallest variations among customers within each group.
The goal of cluster analysis in marketing is to accurately segment customers in order to achieve more effective customer marketing via personalization. A common cluster analysis method is a mathematical algorithm known as k-means cluster analysis, sometimes referred to as scientific segmentation. The clusters that result assist in better customer modeling and predictive analytics, and are also are used to target customers with offers and incentives personalized to their wants, needs and preferences.

In [ ]:
customer_seg = df.groupby('CustomerName').sum().reset_index()
customer_seg = customer_seg[['CustomerName', 'Amount', 'Quantity']]
customer_seg.head()

In [ ]:
# Standardizing
# Code to create 'df' DataFrame (from previous cell or script):
# ... (Code to define and populate the 'df' DataFrame)

# Creating the customer_seg DataFrame
customer_seg = df.groupby('CustomerName').sum().reset_index()
customer_seg = customer_seg[['CustomerName', 'Amount', 'Quantity']]
customer_seg.head()


# Standardizing (code where the error occurred)
customer_seg2 = customer_seg[['Amount', 'Quantity']]
scaler = StandardScaler()
scaler.fit(customer_seg2)

customers_normalized = scaler.transform(customer_seg2)
customers_normalized

# ... (Rest of the code)
scaler = StandardScaler()
scaler.fit(customer_seg2)

customers_normalized = scaler.transform(customer_seg2)
customers_normalized

# Elbow Method to find best number of clusters
sse = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(customers_normalized)
    sse[k] = kmeans.inertia_ # SSE to closest cluster centroid

# Plotting SSE
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(sse.keys()),
    y=list(sse.values()),
    connectgaps=True # override default to connect the gaps
))

fig.update_layout(title_text='The Elbow Method',
                 template='plotly_white')
fig.update_xaxes(title_text='k')
fig.update_yaxes(title_text='SSE')
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True

fig.show()

The k-value of 3 is the best hyperparameter for our model because the next k-value tend to have a linear trend.

In [ ]:
# KMeans
model = KMeans(n_clusters=3)
model.fit(customers_normalized)
customer_seg['Cluster'] = model.labels_ + 1
customer_seg['Cluster'] = customer_seg['Cluster'].astype('category')
customer_seg.head()

,CustomerName,Amount,Quantity,Cluster
0,Aakanksha,74.0,8,1
1,Aarushi,4701.0,49,2
2,Aashna,1931.0,32,3
3,Aastha,3276.0,28,3
4,Aayush,556.0,18,1


In [ ]:
customer_seg.groupby('Cluster').agg({
    'Amount':'mean',
    'Quantity':'count'}).round(2)

<ipython-input-21-d120ca28b1b9>:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Amount,Quantity
Cluster,,
1,452.98,209
2,5596.22,23
3,2081.16,100


In [ ]:
fig = px.scatter(customer_seg, x="Quantity", y="Amount",
                 color="Cluster",
                 template='plotly_white',
                 title="Amount vs Quantity - Customer Segmentation")
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True

fig.show()

**Segment 1**: Medium Buyers <br>
**Segment 2**: Loyal Buyers <br>
**Segment 3**: Occational Buyers <br>
<hr>